In [67]:
import pandas as pd
import seaborn as sns
sns.set_style("whitegrid", {
 'axes.spines.bottom': True,
 'axes.spines.left': True,
 'axes.spines.right': True,
 'axes.spines.top': True
})
sns.set(font_scale=1)
#sns.set_style("darkgrid")
#sns.set_context("poster")
import sys
import os
from scipy.optimize import newton, minimize, fsolve
from scipy.interpolate import InterpolatedUnivariateSpline, krogh_interpolate, PchipInterpolator, interp1d
import numpy as np
import copy
import re
from pathlib import Path

import neutcurve
from neutcurve.colorschemes import CBMARKERS, CBPALETTE

import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
import matplotlib.colors as mcolors
palette = list(mcolors.TABLEAU_COLORS.keys())
palette.extend(['salmon', 'teal','yellowgreen'])
%matplotlib inline

import random
from pyswarm import pso
import isocor

In [68]:
element_natural_prob = {
    'C': 0.0107,
    'H': 0.000115,
    'N': 0.00364
}

In [69]:
### Adjust for specific label:
def obj_iso(v1, element_natural_prob, element_list, print_sim=False):
    # Adjust:
    element_prob = [v1, element_natural_prob[element_list[1]], element_natural_prob[element_list[2]], v1]

    N_trials = 10000000
    ra = np.random.binomial(1, element_prob, (N_trials, len(element_prob)))
    sim_data = np.bincount(ra.sum(1)) / len(ra) * 100
    # Adjust:
    loss = abs(obs_data[2] - sim_data[2]) + abs(obs_data[3] - sim_data[3])
    print(v1, loss)
    if print_sim:
        return(sim_data)
    else:
        return(loss)

In [71]:
# Asp 13C2 position 2,3. m+0,1,2,3,4 abundance data:
metabolite = 'Asp'
ion_formula = 'C4H6NO4'
tracer_element = '13C'
charge = -1
obs_data = {
    0: 0,
    1: 0,
    2: 97.8505095,
    3: 2.1478276,
    4: 0.0016629
}

element_list = ['C']*4

In [72]:
# Correct the effect of natural abundance of other elements than carbon.
# No natural abundance correction for the tracer and assume 100% tracer purity
# to avoid multiplication with the purity matrix.
corrector_HR = isocor.mscorrectors.MetaboliteCorrectorFactory(formula=ion_formula, label=metabolite, tracer=tracer_element, tracer_purity=[0, 1], resolution=240000, mz_of_resolution=200, charge=charge, correct_NA_tracer=False, resolution_formula_code='orbitrap')

In [73]:
# Calculate the corrected isotope fraction:
corrected_area, iso_fraction, res, m_enr = corrector_HR.correct([obs_data[i] for i in range(len(obs_data))])

In [77]:
# Update the data:
for i in range(len(obs_data)):
    obs_data[i] = iso_fraction[i] * 100

In [78]:
# Use particle swarm to find best probability to fit the data:
def iso_fun(v1): return(obj_iso(v1, element_natural_prob, element_list))
xopt, fopt = pso(iso_fun, (0.95,), (0.9999999,), swarmsize=20, maxiter=20, minstep=1e-7, minfunc=1e-7)

[0.97036894] 5.725297099999995
[0.95714428] 8.221557099999984
[0.99079096] 1.799767099999984
[0.96197668] 7.303587099999989
[0.9813954] 3.6274870999999895
[0.96077732] 7.541977099999984
[0.99020404] 1.916107099999985
[0.97749482] 4.37482709999999
[0.96434705] 6.864807099999987
[0.97170061] 5.4788070999999965
[0.97900815] 4.078817099999988
[0.99584469] 0.8195570999999893
[0.95198689] 9.19238709999999
[0.9716179] 5.4881170999999895
[0.98649587] 2.6345570999999794
[0.98645979] 2.6372670999999985
[0.95230721] 9.127827099999994
[0.96951507] 5.869837099999994
[0.9676499] 6.240247099999986
[0.9905176] 1.8533870999999862
[0.95680328] 8.28404709999999
[0.95] 9.562287099999985
[0.9993161] 0.14289709999998124
[0.9914997] 1.6708970999999813
[0.9999999] 0.04075810000000901
[0.96910421] 5.961157099999986
[0.9999999] 0.04623810000001116
[0.9999999] 0.044218100000004146
[0.96772997] 6.217377099999979
[0.98880064] 2.1916770999999855
[0.96312459] 7.109817099999995
[0.99875419] 0.2528570999999915
[0.9615

In [79]:
# Print probability:
xopt[0], fopt

(0.9999999, 0.03916810000000348)

In [80]:
# Get predictions:
obj_iso(xopt[0], element_natural_prob, element_list, print_sim=True)

0.9999999 0.033448100000021075


array([0.00000e+00, 2.00000e-05, 9.78624e+01, 2.12627e+00, 1.13100e-02])

In [81]:
obs_data

{0: 3.91989890898343e-16,
 1: 5.509140396898849e-19,
 2: 97.85050949999999,
 3: 2.1478276,
 4: 0.0016629}